# Tidy data in Pandas

### (An opinionated guide to tabular data analysis in Python)

`git clone --depth=1 git@github.com:ASPP/2019-camerino-tidy-data`

Jupyter reminders:

- IF IN DOUBT: Press Esc (enter command mode), then h (get all keyboard shortcuts)

Some of the most useful commands:

* Switching modes:
    - Esc : get into Command mode (leaves Edit mode)
    - Enter : edit a cell (puts you in Edit mode)

- Ctrl+Enter : run the code in the current cell
- Shift+Enter : run the code in the current cell AND advanced to next cell (works in any mode)

* In command mode:
    - h : see help (see all commands)
    - a / b : add new cell above/below
    - m : turn current cell into a Markdown cell
    - y : turn current cell into a Code cell

You can get help on Python objects and functions with `help()` or the `?` operator.

## Setup 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# This causes Jupyter to display any matplotlib plots directly in the notebook
# It also works for pandas and seaborn, since they use matplotlib to render plots
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

# Schedule(ish)

- 16:00-16:45 basic pandas
- 16:45-17:00 introduction to split-apply-combine
- (break)
- 17:30-18:00 split-apply-combine (ctd)
- 18:00-18:30 plotting (matplotlib, seaborn, Altair)
- 18:30-19:00 tidy data and how to get it

# Basic pandas 🧼🐼

You can find the Pandas website and documentation at https://pandas.pydata.org/

Pandas is a popular Python library for handling tabular data. It provides much of the same functionality for Python as do data frames in the R language. 

The fundamental data types in Pandas are a Series, representing a 1D array of data. Programmatically, a good way to think of a Series is of *a (1D) NumPy array* **plus** an *index*, allowing you to access elements not just by numeric order in the array, but also by the index (could be names, string IDs, timestamps, or even tuples). By default, series indices are numeric 0-indexed, just like NumPy arrays.

In [ ]:
s = pd.Series(["Diego","Jessica","Farah"])
s

and a DataFrame, representing a 2D table of data. Conceptually, a DataFrame is like a spreadsheet or a table in a database. Programmatically, a good way to think of DataFrames is a *dictionary* mapping *column names* to *pandas Series*. That is indeed how to create them.

In [ ]:
df = pd.DataFrame({'Name': ["Diego","Jessica","Farah"],
                   'Age': [34, 27, 50]})
df

There is an important constraint on the Series of a DataFrame: they must all share the same index.

In [ ]:
s.index = 476, 839, 234

In [ ]:
ages = pd.Series([34, 27, 50])
ages.index = 839, 234, 998

In [ ]:
df2 = pd.DataFrame({'name': s, 'age': ages})
df2

Each column of a DataFrame [has a particular type](https://pbpython.com/pandas_dtypes.html) (ints, floats, datetimes, strings etc).

Above, we constructed data manually using lists and dictionaries, to help us understand the pandas data model. For the rest of this workshop, we will work with real data, which mostly means reading from CSV files or Excel files. (But note the existence of [Apache Parquet](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html) for a much more efficient, fast, and data-preserving alternative.)

## Data 

The data we'll use for this section is from the [Long Term Evolution Experiment (LTEE)](http://myxo.css.msu.edu/ecoli/). This experiment has been running for over 30 years (Feb 1988) and over 50,000 E. coli generations, and is still ongoing. It thus firmly belongs in the annals of badass experiments in science.

Twelve separate populations of E. coli have been propagated for the life of the experiment. Every 500 generations, each population is cloned and stored, allowing researchers to study evolutionary behaviour over the long term, and to "rewind and replay" alternate evolutionary trajectories by propagating from an earlier clone. 

Several interesting events have occurred during the experiment. Some populations have spontaneously developed hypermutator phenotypes. In addition, around generation 31,000 one population, Ara-3, spontaneously developed a rare and novel Cit+ mutation, giving it the ability to metabolise citrate in the substrate.

There have been many publications from this experiment. A handful:

- [Blount et al 2008: Historical contingency and the evolution of a key innovation in an experimental population of Escherichia coli](https://www.pnas.org/content/105/23/7899) - on the spontaneous development of citrate metabolisation and on potentiating mutations
- [Tenaillon et al 2016: Tempo and mode of genome evolution in a 50,000-generation experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4988878) - various investigations by sequencing and variant-calling over 50,000 generations of clones, including discussion of hypermutator phenotypes and genetic drift vs natural selection.

Sequence data from clones is available, but for this workshop we'll be using some published tabular data.

A version of this dataset is also used by the [Data Carpentry lessons on Genomics](https://datacarpentry.org/genomics-workshop/).

In the repository you'll find the files:

- ltee_sampleruns.csv : sample and sequencing run metadata for the E. coli clones
- ltee_mutations.csv : analysis output from variant calling on the E. coli clones
- ltee_relative_fitness.tsv : relative fitness values for each population and generation up to generation 10,000
- ltee_cell_size.tsv : cell sizes for each population and generation up to 10,000

Let's read in the sample and run metadata. Pandas has functions for reading in many data types. Try looking at the documentation for `read_csv()` by running `help(pd.read_csv)` or `pd.read_csv?`.

In [ ]:
sampleruns = pd.read_csv('data/ltee_sampleruns.csv')

In [ ]:
# The number of rows and columns
sampleruns.shape

In [ ]:
# The first few rows
sampleruns.head()

In [ ]:
# Column data types
sampleruns.dtypes

In [ ]:
# Column names
sampleruns.columns

In [ ]:
# Index (i.e. row names)
# In this case we didn't provide an index and rows have simply been numbered for us by Pandas
sampleruns.index

### Exercise 1

1. Use `pd.read_csv()` to read the file `ltee_mutations.csv` into a variable called `mutations`.
2. Check the column headings and the number of rows in this dataset, and have a look at the first few rows. Compare the size of the dataset and the variables to `sampleruns`.

In [ ]:
# play here...

In [ ]:
# ... then write your solution into this file and load it.
%load pandas_and_tidying_ex1.py

## Indexing and slicing 

Just like a dictionary, we can extract a column from the DataFrame by indexing with square brackets, e.g. 

In [ ]:
# Extracting a column
sampleruns['Strain ID']

Question: what is the type of `sampleruns['Strain ID']`?

Firstly, let's set our index (row names) to something more meaningful to make it easier to see what's going on. The Strain ID uniquely identifies each sample, so it is probably a sensible index. We can use `sampleruns.set_index()`, or we can assign to the index directly:

In [ ]:
sampleruns.index = sampleruns['Strain ID']
sampleruns.head()

The two most important ways to extract data from a DataFrame are `loc` and `iloc`. `loc` uses the index and the column names; `iloc` uses the row and column numbers, counting from zero. 

In [ ]:
sampleruns.loc['REL768B', 'Accession']

In [ ]:
# Row 0, column 9
sampleruns.iloc[0, 9]

In [ ]:
# Rows 1-3, column 9
sampleruns.iloc[1:4, 9]

In [ ]:
# All columns
sampleruns.loc['REL768B', :]

In [ ]:
# Lists let us specify any set of rows and columns, in any order
sampleruns.loc[['REL768A','REL958A'], ['Read Type', 'Read Length']]

In [ ]:
# We can also use True/False values to perform boolean indexing. 
# Pandas will return the rows/columns matching the True values we pass in.
# This will be useful later for filtering data
iris.loc[0:5, [True, False, True, True, False]]

### Exercise 2

1. Set the index of `mutations` to be the same as the "Strain ID" column.
2. Extract the population, generation, and number of total mutations for strain REL11345

In [ ]:
# play here...


In [ ]:
# ... then write your solution into this file and load it
%load pandas_and_tidying_ex2.py

## Manipulating Series

A single column of a DataFrame is a Series object. Series have a data type:

In [ ]:
sampleruns['Sequencing Depth'].dtype

Like a DataFrame, a Series has an index. In this case we got our Series from a column of a DataFrame, so it will have the same index:

In [ ]:
sampleruns['Sequencing Depth'].index

There are several convenience functions defined on Series. For instance, we can find the average sequencing depth:

In [ ]:
sampleruns['Sequencing Depth'].mean()

Similarly for numeric variables we have, for instance, `.min()` and `.max()`, `.median()`, `std()`, and `sum()`.

`.describe()` is a convenience function for getting several summary statistics at once:

In [ ]:
sampleruns['Sequencing Depth'].describe()

For non-numeric variable types such as strings and categoricals, we may want to look at the unique values.

In [ ]:
sampleruns['Read Type'].unique()

In [ ]:
sampleruns['Read Type'].value_counts()

Pandas makes use of numpy vectorisation, meaning we can do operations on Series with simple syntax, and it will be efficient to run.

In [ ]:
# Assuming 500 generations takes 75 days
mutations['Days'] = mutations['Generation'] * 0.15
mutations.head()

In [ ]:
mutations['Population'].value_counts()

In [ ]:
(mutations['Total Deleted Base Pairs'] + mutations['Total Inserted Base Pairs']).head()

You might have noticed that the `Analysis Notes` column contains a lot of NaN's. This means "not a number" and represents a missing value - i.e. these cells are empty. 

In [ ]:
sampleruns.head()

We can check which values are missing with `.isnull()`. This converts every value in the DataFrame (or Series) into a boolean True/False value.

In [ ]:
sampleruns.isnull().head()

Adding up booleans will treat `True` as `1` and `False` as `0`. A common approach is to use `sum()` to count how many `True` values there are. So we can count missing values like this:

In [ ]:
sampleruns.isnull().sum()

`sampleruns` had 264 rows, so it looks like there are a few non-empty note cells. We could count this explicitly by taking the logical `not` of our True/False values, i.e. adding up cells where `isnull()` is `False`. For manipulating array-like data, we can't use the `not`, `and` and `or` boolean operators. Instead we need to use the bitwise operators `~`, `&`, and `|`.

In [ ]:
(~sampleruns.isnull()).sum()

Notice that Pandas summed each column. We can use `sum(axis=1)` to override this default and sum each row instead.

## Sorting and filtering 

We can sort on a field, or list of fields, with `.sort_values()`:

In [ ]:
# Get a random example subset
subset = sampleruns.sample(15)
subset

In [ ]:
subset.sort_values('Generation')

In [ ]:
subset.sort_values(['Population','Generation'])

We can filter datasets using boolean indexing. This means that if we use a logical expression produce a boolean Series with a logical expression:

In [ ]:
subset['Population'] == 'Ara+5'

we can then select out only the rows (or sometimes columns) where that logical expression is True:

In [ ]:
subset.loc[subset['Population'] == 'Ara+5', :]

### Exercise 3

1. Filter the `sampleruns` dataset to extract only rows which contain Analysis Notes, i.e. those where this field is not empty.
2. Filter the `mutations` dataset to extract only samples with more than 1500 total mutations. 
3. Sort the resulting data from (2) by the number of "Small Indels". Have a look at the resulting Population and Generation columns.


In [ ]:
# play here...


In [ ]:
# ... then write your solution into this file and load it
%load pandas_and_tidying_ex3.py

## Merging data 

### Demo data 

We'll read in the classic `iris` and `mtcars` datasets for demo purposes.

In [ ]:
iris = pd.read_csv('data/iris.csv')
cars = pd.read_csv('data/mtcars.csv')

We can merge two datasets together by matching corresponding variables.

Our main options are `DataFrame.join()` and `pandas.merge()`. `merge()` is a little more flexible, so we'll demonstrate that.

Recall the `cars` dataset:

In [ ]:
cars.head()

The `origin` column here, which is a number 1-3, is actually intended to represent the country of origin. It's encoded as:

- USA : 1
- Europe : 2
- Japan : 3

Let's make a DataFrame to represent this mapping. We'll add a fourth code for Australia, which doesn't appear in the dataset!

In [ ]:
origin_codes = pd.DataFrame(
    {'origin': [1, 2, 3],
     'origin_country': ['USA', 'Europe', 'Japan']}
)

origin_codes

We can use `pandas.merge()` to join our `cars` table to our `origin_codes` table using the shared `origin` field, like so:

In [ ]:
cars_ori = pd.merge(cars, origin_codes)
cars_ori.sample(10)

This "just worked" because Pandas correctly deduced that the identically-named field(s) were the ones to match on. Sometimes we might need to be more verbose. In this case, this accomplishes the same thing as the above:

In [ ]:
# use origin from the left dataframe (cars) and from the right (origin_codes)
# use how="left" (keep all origin values that exist in the left dataframe)
cars_ori = pd.merge(
    cars,
    origin_codes,
    left_on='origin',
    right_on='origin',
    how='left',
)

### Exercise 4

Try to merge our `sampleinfo` and `mutations` columns. This time there are three shared fields: 'Strain ID', 'Population', and 'Generation'. (*Hint:* although only 'Strain ID' is needed to uniquely identify  rows, we want to specify all matching variables so that Pandas knows to only include each of these variables once in the resulting dataframe.)

In [ ]:
# play here...


In [ ]:
# ... then write your solution into this file and load it
%load pandas_and_tidying_ex4.py

# Split-apply-combine

<img src="https://miro.medium.com/max/1170/1*w2oGdXv5btEMxAkAsz8fbg.png" />

(Source: https://medium.com/analytics-vidhya/split-apply-combine-strategy-for-data-mining-4fd6e2a0cc99)

In [ ]:
cars.groupby('cylinders')['mpg'].mean()

### Exercise 5

Now that the metadata on Mutator phenotypes is together with the information on actual mutations, we can try exploring the relationships between these fields. What is the average number of mutations for each mutator phenotype?

In [ ]:
# play here...


In [ ]:
# ... then write your solution into this file and load it
%load pandas_and_tidying_ex5.py

## Writing to a file

To finish off, let's write out our tidied and merged table to a new file, for future analyses. To write to CSV, we can use the `to_csv` method.

In [ ]:
# Don't keep the index as we still have the Strain ID column
ltee.to_csv('ltee_solution.csv', index=False)

## Publication costs

In [ ]:
subs = pd.read_csv('data/publishers.csv')

In [ ]:
subs.head()

Source: Stuart Lawson, [Subscription costs FOIs](https://figshare.com/articles/Journal_subscription_costs_FOIs_to_UK_universities/1186832)

### Exercise 6

Explore the following questions using subsetting and split-apply-combine:

- what was the total cost of publishing to UK research (among public universities) in the years 2010-2016?
- what was the total cost of publishing to UK research in 2016?
- How much did universities pay to Elsevier in each of the years 2010-2016?
- *(Advanced)* Use a [named aggregation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#named-aggregation) to find the min/max/mean/median that universities paid to Elsevier in 2016.

In [ ]:
# play here...


In [ ]:
# ... then write your solution into this file and load it
%load pandas_and_tidying_ex6.py

# Plotting data

## Why visualize data

- Exploration
- Exposition
- Design

## Exploration

<img src="https://d2f99xq7vri1nk.cloudfront.net/DinoSequentialSmaller.gif" />

(source: https://www.autodeskresearch.com/publications/samestats)

## Exposition

<img src="_images/viz.png" width=250/>

## Data encodings

| A     | B      |
|-------|--------|
| 6.61  | 16.81  |
| 7.89  | 16.84  |
| 10.54 | 18.96  |
| 11.30 | 18.98  |
| 12.27 | 20.96  |
| 12.27 | 20.97  |

<img src="_images/encoding.png"/>

## Categorical point encodings

<img src="https://media.springernature.com/m685/nature-assets/nmeth/journal/v10/n6/images/nmeth.2490-F2.jpg" />

Source: Krzywinski & Wong, Plotting symbols, Nature Methods 2013

## Gestalt principles

http://emeeks.github.io/gestaltdataviz/section1.html

### Proximity

![Great wall](_images/great-wall.png)

### Proximity and similarity

![asteroid belt](_images/asteroids.png)

Source: Jake VanderPlas, 2013 John Hunter Excellence in Plotting competition, 3rd place

## Data 

For this section we're going to load in two main datasets:

### House sales data 

This data contains information about house sales in Seattle in 2014 and 2015. We have information on the houses themselves: location, size, quality, view, and whether the house is tagged as 'waterfront' or not. We also have information on the date and price of each sale.

In [ ]:
sales = pd.read_csv(
    'data/housing-data-10000.csv', 
    usecols=['id','date','price','lat','long', 'zipcode',
             'waterfront','view','grade','sqft_living'],
    parse_dates=['date'], 
    dtype={'zipcode': 'category',
           'waterfront': 'bool'},
)

## Matplotlib 

Matplotlib is the oldest and still the fundamental plotting library in Python. It has a huge range of capabilities. Many other libraries (including Seaborn) use Matplotlib as a back-end renderer.

Today we're focussing on plotting tabular data. We won't touch on all Matplotlib's capabilities. If you want to see more of the range of things Matplotlib can do, you can look through the [Matplotlib gallery](https://matplotlib.org/gallery.html.), or try out this excellent [Matplotlib tutorial](https://www.labri.fr/perso/nrougier/teaching/matplotlib/).

An example Matplotlib plot with legend and annotation:

In [ ]:
x = [1,2,3,4,5]
y = [2,5,10,17,26]
y2 = [1,4,9,11,9]

fig, ax = plt.subplots()
ax.plot(x, y, c='blue', label='Projected')
ax.scatter(x, y2, c='red', label='Actual')
fig.legend()
ax.annotate(
    'where it all went wrong', 
    xy=(3,10),
    xytext=(1,12),
    arrowprops={'width': 2}
)

fig.savefig('example_matplotlib.png')

## Seaborn 

Seaborn builds on Matplotlib. Some nice features are:

- works directly with Pandas dataframes, concise syntax
- lots of plot types, including some more advanced options
- statistical plotting: many plots do summary statistics for you
- good default aesthetics and easy control of aesthetics
- uses Matplotlib, so can use all Matplotlib backends (incl lots of image file formats)
- underlying Matplotlib objects can be tweaked directly

For completeness, here's the plot we made before:

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9]
})

fig, ax = plt.subplots()
sns.scatterplot(data=df, x='Time', y='Actual', color='red', ax=ax)
sns.lineplot(data=df, x='Time', y='Projected', color='blue', ax=ax)

ax.annotate(
    'where it all went wrong', 
    xy=(3,10),
    xytext=(1,12),
    arrowprops={'width': 2}
)

Notice that we can add changes like annotations in exactly the same way, as we have Matplotlib Figure and Axes objects.

### Seaborn and Pandas 

In some cases we can use Seaborn by passing in lists (or arrays or series) directly:

In [ ]:
sns.barplot(x=['A','B','C'], y=[33,44,20])

However Seaborn is aware of Pandas and it is very common to use Seaborn directly with DataFrames. Plotting functions can take a DataFrame as their `data` parameter and then refer directly to column names:

In [ ]:
sns.barplot(data=iris, x='species', y='petal_length')

Here Seaborn has interpreted the `x` and `y` arguments as field names in the supplied DataFrame. Notice also that Seaborn has performed the summary statistics for us - in this case, using the default `estimator`, which is `mean()`. 

Notice also what happens if we simply swap the `x` and `y` parameters. Seaborn will automatically deduce that the categorical or string-like variable must be the bar labels, and the numeric variable must be the numeric axis:

In [ ]:
sns.barplot(data=iris, y='species', x='petal_length')

What happens if the grouping is not obviously categorical? How could we fix this?

In [ ]:
sns.barplot(data=cars, x='acceleration', y='origin')

### Exercise 7: 

**1:** Create a count plot using `sns.countplot()` on the `ltee` data, showing how many clones have each `Mutator` phenotype. Note that you do not need to specify the `y` axis variable for a countplot, just the `x` axis variable (i.e. category).

**2:** Create a (vertical) bar plot using the `sales` data, showing how house prices vary with the value of the property `grade`.

Bar plots are often deplored as a way of showing statistical estimates, as only the top of the bar is really important, and the bar itself is a visual distraction. A point plot is an alternative, and plots like box plots can show more information. Several other plot types also show distributional information within categories.

**3:** Reproduce the plot you just made, using instead each of the Seaborn functions:

- pointplot()
- boxplot()
- stripplot() [SEE WARNING]  (try the `jitter` parameter)
- swarmplot() [SEE WARNING]

Note what sort of information about the distribution is shown by each.

WARNING: `stripplot()` and `swarmplot()` will plot individual data points. There are too many house sales to easily display in this way - you should subsample the dataframe with e.g.  `data=sales.sample(100)`.

### Hue 

Many Seaborn plotting functions take a `hue` parameter. This colours the plot elements by some categorical variable, but more than this, summary statistics are calculated for each level of the hue variable.

In [ ]:
sns.scatterplot(data=ltee, 
                x='Generation', 
                y='Synonymous Base Substitutions', 
                palette='bright') # hue='Population')

In [ ]:
sns.lineplot(data=ltee, 
             x='Generation', 
             y='Synonymous Base Substitutions', 
             palette='bright') # hue='Population')

### Exercise 8:

- Using the subscription costs dataset (`subs`), plot the costs over time for each publisher, using the "hue" parameter to "lineplot" or "barplot".
- Use the underlying matplotlib axes object and it's `set_scale` method to set a log scale on the cost.

### Colour and Palettes

Seaborn has good colour options. There are a few ways we could want to use access colours:

* Specify an individual colour for some plot element. Matplotlib named colours can be used, or rgb values specified. Also check out the `sns.xkcd_rgb` dictionary of 954 named colours from the XKCD colour survey - for instance, `sns.xkcd['fire engine red']` is a colour.
* Specify a colormap, for mapping a continuous value to colour. All Matplotlib colormaps can be used by name. You can see these under the `plt.cm` module. Seaborn's `light_palette()` and `dark_palette()` functions can also generate custom colourmaps easily.
* Specify a discrete colour palette (a list of colours), for mapping a discrete or categorical variable to colour. In Seaborn, there is a distinction between colour palettes and colormaps. In general, you can create a colour palette by explicitly listing some colours, or by selecting a series of colours along some colormap. There are several functions, including `color_palette()`, `light_palette()`, `dark_palette()`, `diverging_palette()` and `xkcd_palette()`, which can produce many discrete colour palettes of whatever size you need. 

In [ ]:
# An example discrete colour palette of 7 colours, based on the XKCD colour "denim blue"
# palplot is a function to visualise a palette
palette = sns.light_palette("denim blue", n_colors=7, input='xkcd')
sns.palplot(palette)

In [ ]:
# Equivalently (to illustrate that we can use an rgb value directly)
denim_blue = sns.xkcd_rgb["denim blue"]
print(denim_blue)
palette = sns.light_palette(denim_blue, n_colors=7)
print(palette)
sns.palplot(palette)

# Altair

Altair is not bundled with Anaconda - you may need

`conda install -c conda-forge altair vega`

or

`pip install altair vega`

In [ ]:
# import and initialise altair
import altair as alt
# this line is needed in jupyter notebook, but not jupyter-lab
alt.renderers.enable('notebook')

## Data 

### LTEE data 

In this lesson we'll use a large flat file containing both sample metadata on each clone, and information on observed mutations in their genomes.

In [ ]:
# If you have done the pandas and data tidying workshop and saved the final file,
# you can try reading that file in here if you prefer
ltee = pd.read_csv(
    'data/ltee_merged.csv', index_col='Strain ID'
)

### House sales data 

We'll use the house sales data from above, but we will subsample it for Altair, which is not (yet) designed for large datasets.

In [ ]:
ssales = sales.sample(3000)

[Altair](https://altair-viz.github.io) is a library for creating interactive plots. 

Altair is built around the [Vega-Lite](https://vega.github.io/vega-lite/) schema, a "visualisation grammar". Altair plots are specified in Python, then converted behind the scenes to a declarative JSON structure that follows the Vega-Lite schema, which can then be rendered by a Javascript library.

Altair works very well with Pandas - in fact, it usually expects data to be in a Pandas DataFrame.

### Simple interactive plotting

Here's a simple example of an Altair plot.

In [ ]:
df = pd.DataFrame(
    {'Time': [1,2,3,4,5],
     'Projected': [2,5,10,17,26],
     'Actual': [1,4,9,11,9]},
)

chart = alt.Chart(df)

chart.mark_line(color='blue').encode(x='Time', y='Projected')

Here `mark_line()` tells Altair we want to draw a line, and `encode()` is used to assign our variables (columns of the DataFrame) to possible encoding channels of the line. We've just used `x` and `y`. Notice that we set the colour in `mark_line()`; if we'd wanted to set the colour to encode some variable, we would have set it in `encode()`.

Simple pan-and-zoom interactivity can be added with `.interactive()`:

In [ ]:
chart = alt.Chart(df)

chart.mark_line(color='blue').encode(x='Time', y='Projected').interactive()

If we wanted to produce the chart we produced earlier, with scatter points, we can use `alt.layer()`, which takes the components as arguments and layers them on top of each other.

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9],
})

chart = alt.Chart(df)
line = chart.mark_line(color='blue').encode(
    x='Time', y='Projected'
)
points = chart.mark_point(color='red').encode(
    x='Time', y='Actual'
)

alt.layer(line, points).interactive()
# or (line + points).interactive()

### Exercise 9: 

Recreate, in Altair, the scatter plot of house sales with x-coordinates given by `longitude` and y-coordinates given by `latitude`. You can use `.mark_point()` or `.mark_circle()`. One issue you will discover, if you use the `x` and `y` encodings, is that Altair and Vega-lite include the axis zero by default. If you have time, do some googling to see how you can fix that.

## Data types 

Altair recognises four fundamental [data types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types):

| Data Type | Shorthand Code | Description |
|-|-|-|
| quantitative | Q | a continuous real-valued quantity |
| ordinal | O | a discrete ordered quantity |
| nominal | N | a discrete unordered category |
| temporal | T | a time or date value |

In [ ]:
(alt.Chart(ssales.sample(10))
    .mark_bar()
    .encode(x='date:N', y='price:Q')
)

In [ ]:
# What kind of sequencing experiments have been done on the LTEE data?

chart = alt.Chart(data=ltee)

(chart.mark_point()
    .encode(x=alt.X('Sequencing Depth', 
                    scale=alt.Scale(type='log')),
            y='Read Type',
            color='Read Length:N')  # try N, O, Q
)

### Exercise 10:

* Colour your "geographical" scatter plot of house sales according to whether the property is or is not a waterfront property. What is the appropriate data type?
* Colour according to price (if you prefer, try a log scale). What is the appropriate data type?
* Colour according to view. What is the appropriate data type?

## Marks and encodings 

Altair has various marks used to draw different kinds of plots. For instance:

* `mark_point()` : points on a scatter plot
* `mark_bar()` : rectangular bars, in e.g. a bar plot or histogram
* `mark_area()` : filled/shaded areas

The full list of marks can be found at [https://altair-viz.github.io/user_guide/marks.html](https://altair-viz.github.io/user_guide/marks.html).

We can assign variables of the DataFrame to different encoding channels. For instance:

* `x` and `y` : position of mark (scatterpoint position, top of bar-chart bar, etc)
* `color` : colour of the mark (colour of point, bar, shaded area etc)
* `size` : size of the mark (point size, bar width, etc)

You can find a list of encodings at [https://altair-viz.github.io/user_guide/encoding.html](https://altair-viz.github.io/user_guide/encoding.html).

In [ ]:
# Demo: marks from common base
base = (
    alt.Chart(ssales.sample(10))
       .encode(x='date:T', y='price:Q')
)

alt.hconcat(base.mark_point(color='red'), base.mark_bar())
# or base.mark_point(color='red') | base.mark_bar()

In [ ]:
# Demo: box plot
base = alt.Chart(ltee)

base.mark_boxplot().encode(
    x='Population', y='Total Mutations'
)
# Log scale works here
# Color also works here (sort of!)

## Interactivity 

Hover text can be added to a plot by setting the `tooltip` encoding.

### Exercise 11: 

On your scatter plot of house location, set the hover info to display the sale price.

Basic pan-and-zoom interactivity is set with `.interactive()`. This is actually short for:

```
chart.add_selection(
    alt.selection_interval(bind='scales')
)
```

This means we've added a behaviour where dragging the mouse (selecting an interval) causes the plot to zoom and pan (bind='scales'). We can create custom interactive behaviour with [bindings, selections, and conditions](https://altair-viz.github.io/user_guide/interactions.html).


# Tidy data

<img src="https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png" />

(source: https://r4ds.had.co.nz/tidy-data.html#fig:tidy-structure)

Open the "house-sales" excel spreadsheet.

## Reshaping data 

We can reshape data using particular columns, with `melt` and `pivot` or `pivot_table`. We'll have a look at this below.

We can also reshape data using the column names and index, with `stack` and `unstack`. This requires MultiIndexes, which we won't go into today.

Here are two tiny "wide" datasets based on our "untidy" housing data:

In [ ]:
sales_wide1 = pd.read_excel(
    'house-sales.xlsx',
    sheet_name='untidy data',
    header=0,
    usecols='B:J',
    index_col=0,
    skiprows=list(range(4)),
    nrows=3,
    parse_dates=False,
)

In [ ]:
sales_wide1

Unfortunately, Excel has already stuck its grimy fingers into the data integrity pie, and converted some text to dates, inaccurately in this case. The only way to fix this is manually.

In [ ]:
sales_wide1['sale date 1'] = pd.to_datetime(['2014-10-16', '2014-09-24', '2014-12-09'])

In [ ]:
sales_wide1

In [ ]:
sales_wide1['sale date 2'] = pd.to_datetime([None, '2014-12-15', None])

In [ ]:
sales_wide1

### Exercise 12

- Use `pd.read_excel` as above to read the 2015 house sales table into a DataFrame.
- Ensure the sale dates are accurate datetimes.

In [ ]:
# sales_wide2 = ...

In [ ]:
sales_wide2['sale date 1'] = pd.to_datetime(sales_wide2['sale date 1'])

In [ ]:
sales_wide2['sale date 1']

In [ ]:
sales_wide2['sale date 2'] = pd.to_datetime([None, '2015-04-30', None, None])

How can we reshape these into tidy form? 

The Pandas `melt` function will do this. For instance:

In [ ]:
sales_wide1.melt()

This has put every variable (i.e. every column) into the new `variable` column. This probably isn't what we want. It's only the price columns that are "wide", the other variables were fine.

In [ ]:
# Retain columns property_id and bedrooms
sales_wide1.melt(id_vars=['property_id','bedrooms'])

This is getting close to what we want. The `variable` column contains the original column names and tells us whether the price we're looking at was from the first or second sale (this may or may not be information we care about). The `value` column contains values in the melted columns, i.e. the actual price. 

Now we technically have long form and have eliminated the duplicated `price` variable; all prices are now in the `value` column. Notice that properties can now appear more than once in the table; conceptually, we have a row per sale rather than a row per property. 

We can tell `melt()` what to call the `variable` and `value` columns:

In [ ]:
sales_tidy = sales_wide1.melt(
    id_vars=['property_id','bedrooms'], 
    var_name='sale_number',
    value_name='price',
)

We still have empty extra rows where there was no sale in the original table, i.e. rows 3 and 5. We could use `dropna()` to get rid of these. A more generic approach would be to use filtering:

In [ ]:
sales = sales_wide1.melt(
    id_vars=['property_id','bedrooms'], 
    var_name='sale_number',
    value_name='price')
sales = sales[~sales['price'].isnull()]
sales

This is pretty good! Now we could run commands like `sales["price"].mean()` and get a sensible answer. We'll also be able to use the data easily to produce plots.

If you want a challenge, think about how you could convert `sales_wide2` to tidy form - it's a fair bit harder.

The inverse operation to `.melt()` is `.pivot()`:

In [ ]:
sales.pivot(index='property_id', columns='sale_number',
            values='price')

### Exercise 13

The original Excel spreadsheets from the journal subscription costs tidy table are in:

- Journal_publishing_cost_FOIs_UK_universities.xlsx
- Journalsubscost20152016v6.xlsx

This is the raw data as it was input by the researchers.

- Is it tidy? (Hint: see second part of the exercise ;)
- Use pandas.read_excel, pandas.DataFrame.melt, and pandas.concat to create a complete (2010-2016) tidy dataset from the two files.